# Sources
* BiLSTM model architecture based on [Ozols et. al., 2021](https://www.mdpi.com/1422-0067/22/6/3071/htm)
* ESM2 taken from [Lin et al., 2022](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1), [Github](https://github.com/facebookresearch/esm)

In [1]:
import os
import sys
import csv
import math
import pickle
import random
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, "r") as csvfile:
        train_data = list(csv.reader(csvfile))[1:]  # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls

def apply_random_masking(seq, num_tokens):
    """
    Mask `num_tokens` as 3 (i.e. <unk>) at random positions per sequence.
    """
    dist = torch.rand(seq.shape)
    m, _ = torch.topk(dist, num_tokens)
    return seq * (dist < m) + (dist == m) * 3


def regularized_auc(train_auc, dev_auc, threshold=0.0025):
    """
    Returns development AUC if overfitting is below threshold, otherwise 0.
    """
    return dev_auc if (train_auc - dev_auc) < threshold else 0


def save_metrics(*args, path):
    if not os.path.isfile(path):
        with open(path, "w", newline="\n") as f:
            f.write(
                ",".join(
                    [
                        "fold",
                        "epoch",
                        "train_loss",
                        "train_acc",
                        "train_auc",
                        "val_loss",
                        "val_acc",
                        "val_auc",
                    ]
                )
            )
            f.write("\n")
    if args:
        with open(path, "a", newline="\n") as f:
            f.write(",".join([str(arg) for arg in args]))
            f.write("\n")
            
def gelu(x):
    """
    Facebook Research implementation of the gelu activation function.
    
    For information: OpenAI GPT's gelu is slightly different
    (and gives slightly different results):
    0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))
            
def total_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def trainable_model_params(model):
    return sum(
        p[1].numel()
        for p in model.named_parameters()
        if p[1].requires_grad and not p[0].startswith("esm")
    )

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl

    def __getitem__(self, idx):
        return self.lbl[idx], self.seq[idx]

    def __len__(self):
        return len(self.lbl)


class TrainBatch:
    def __init__(self, batch):
        lbls, _, seq = tokenizer(batch)
        self.seq = apply_random_masking(seq, num_tokens=1)
        self.lbl = torch.tensor([int(l) for l in lbls], dtype=torch.float)

    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self


def train_wrapper(batch):
    return TrainBatch(batch)


class EvalBatch:
    def __init__(self, batch):
        lbls, _, self.seq = tokenizer(batch)
        self.lbl = torch.tensor([int(l) for l in lbls], dtype=torch.float)

    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self


def eval_wrapper(batch):
    return EvalBatch(batch)

In [5]:
class ESM2BiLSTM(nn.Module):
    def __init__(self, esm2, rnn_size, hidden_size, dropout):
        super().__init__()

        self.esm2_encoder = esm2

        self.dropout = nn.Dropout(dropout)

        self.lstm = nn.LSTM(
            input_size=640,
            hidden_size=rnn_size,
            bidirectional=True,
            batch_first=True,
        )

        self.fc1 = nn.Linear(rnn_size * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, seq):
        with torch.no_grad():
            # input shape: (batch_size, seq_len=10+2(cls, eos))
            # out: (batch_size, seq_len, embedding_dim=1280)
            result = self.esm2_encoder(seq, repr_layers=[30])
        
        embedded = self.dropout(result['representations'][30][:, 1:10+1, :])

        # input shape: (batch_size, seq_len, embedding_dim)
        out, _ = self.lstm(embedded)

        # input shape: (batch_size, seq_len=1, 2*rnn_size)
        pooled, _ = torch.max(out, dim=1)

        # input shape: (batch_size, 2*rnn_size)
        out = self.dropout(gelu(self.fc1(pooled)))

        # input shape: (batch_size, hidden_size)
        # output shape: (batch_size)
        return self.fc2(out).squeeze()

In [6]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
    return epoch_loss / total, num_correct / total, roc_auc_score(lbls, preds)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 512

# load train+dev set, mix it back as one
train_path = '../../data/c_train.csv'
dev_path = '../../data/c_val.csv'
test_path = '../../data/c_test.csv'

# combine previously split train and dev set
train_seqs, train_lbls = read_data(train_path)
dev_seqs, dev_lbls = read_data(dev_path)
total_seqs, total_lbls = np.array(train_seqs + dev_seqs), np.array(train_lbls + dev_lbls)

assert len(train_seqs) + len(dev_seqs) == len(total_seqs)
assert len(train_lbls) + len(dev_lbls) == len(total_lbls)

test_seqs, test_lbls = read_data(test_path)

test_data = CleavageDataset(test_seqs, test_lbls)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

# load pre-trained esm2 model and vocab
esm2, vocab = torch.hub.load('facebookresearch/esm:main', 'esm2_t30_150M_UR50D')
tokenizer = vocab.get_batch_converter()

Using cache found in /home/ingo/.cache/torch/hub/facebookresearch_esm_main


In [8]:
NUM_EPOCHS = 10
BATCH_SIZE = 512
RNN_SIZE = 512
HIDDEN_SIZE = 128
DROPOUT = 0.5
LEARNING_RATE = 3e-4

params = {
    "esm2": esm2,
    "rnn_size": RNN_SIZE,
    "hidden_size": HIDDEN_SIZE,
    "dropout": DROPOUT,
}

In [9]:
LEARNING_RATE = 3e-4
criterion = nn.BCEWithLogitsLoss()

kf = KFold(n_splits=5, shuffle=True, random_state=1234)
logging_path = "../../params/c_term/ESM2BiLSTM/results.csv"

start = time()
print("Starting Cross-Validation.")
highest_val_auc = 0

# get a new split
for fold, (train_idx, dev_idx) in enumerate(kf.split(total_seqs), 1):
    X_tr = total_seqs[train_idx]
    y_tr = total_lbls[train_idx]
    X_dev = total_seqs[dev_idx]
    y_dev = total_lbls[dev_idx]

    # create datasets and loads with current split
    train_data = CleavageDataset(X_tr, y_tr)
    train_loader = DataLoader(
        train_data,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=train_wrapper,
        pin_memory=True,
        num_workers=10,
    )

    dev_data = CleavageDataset(X_dev, y_dev)
    dev_loader = DataLoader(
        dev_data,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=eval_wrapper,
        pin_memory=True,
        num_workers=10,
    )

    # reset model weights with each new fold
    if 'model' in globals():
        del model
        print('deleted model')
    if 'esm2' in globals():
        del esm2
        print('deleted esm2')
    esm2, _ = torch.hub.load('facebookresearch/esm:main', 'esm2_t30_150M_UR50D')
    model = ESM2BiLSTM(**params).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # normal training loop
    for epoch in range(1, NUM_EPOCHS + 1):
        model.train()
        train_loss, train_acc, train_auc = process(
            model, train_loader, criterion, optimizer
        )

        model.eval()
        with torch.no_grad():
            val_loss, val_acc, val_auc = process(model, dev_loader, criterion)

        # save metrics
        save_metrics(
            fold,
            epoch,
            train_loss,
            train_acc,
            train_auc,
            val_loss,
            val_acc,
            val_auc,
            path=logging_path,
        )

        print(
            f"Training:   [Fold {fold:2d}, Epoch {epoch:2d}, Loss: {train_loss:8.6f}, Acc: {train_acc:.4f}, AUC: {train_auc:.4f}]"
        )
        print(f"Evaluation: [Fold {fold:2d}, Epoch {epoch:2d}, Loss: {val_loss:8.6f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}]")
        
        reg_auc = regularized_auc(train_auc, val_auc, threshold=0)
        if reg_auc > highest_val_auc:
            highest_val_auc = reg_auc
            path = f"../../params/c_term/ESM2BiLSTM/auc{reg_auc:.4f}_fold{fold}_epoch{epoch}.pt"
            torch.save(model.state_dict(), path)

print("Finished Cross-Validation.")
train_time = (time() - start) / 60
print(f"Cross-Validation took {train_time} minutes.")

Starting Cross-Validation.
deleted esm2


Using cache found in /home/ingo/.cache/torch/hub/facebookresearch_esm_main


Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:06<00:00,  7.56batches/s]
Training:   [Fold  1, Epoch  1, Loss: 0.000734, Acc: 0.8409, AUC: 0.7993]
Evaluation: [Fold  1, Epoch  1, Loss: 0.000638, Acc: 0.8575, AUC: 0.8655]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:05<00:00,  7.61batches/s]
Training:   [Fold  1, Epoch  2, Loss: 0.000667, Acc: 0.8520, AUC: 0.8461]
Evaluation: [Fold  1, Epoch  2, Loss: 0.000622, Acc: 0.8624, AUC: 0.8708]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:06<00:00,  7.57batches/s]
Training:   [Fold  1, Epoch  3, Loss: 0.000656, Acc: 0.8547, AUC: 0.8521]
Evaluation: [Fold  1, Epoch  3, Loss: 0.000615, Acc: 0.8642, AUC: 0.8739]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:05<00:00,  7.61batches/s]
Training:   [Fold  1, Epoch  4, Loss: 0.000649, Acc: 0.8559, AUC: 0.8557]
Evaluation: [Fold  1, Epoch  4, Loss: 0.000613, Acc: 0.864

Using cache found in /home/ingo/.cache/torch/hub/facebookresearch_esm_main


Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:03<00:00,  7.88batches/s]
Training:   [Fold  2, Epoch  1, Loss: 0.000728, Acc: 0.8416, AUC: 0.8041]
Evaluation: [Fold  2, Epoch  1, Loss: 0.000636, Acc: 0.8587, AUC: 0.8639]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:03<00:00,  7.88batches/s]
Training:   [Fold  2, Epoch  2, Loss: 0.000666, Acc: 0.8518, AUC: 0.8465]
Evaluation: [Fold  2, Epoch  2, Loss: 0.000626, Acc: 0.8620, AUC: 0.8690]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:03<00:00,  7.87batches/s]
Training:   [Fold  2, Epoch  3, Loss: 0.000656, Acc: 0.8542, AUC: 0.8520]
Evaluation: [Fold  2, Epoch  3, Loss: 0.000619, Acc: 0.8630, AUC: 0.8722]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:03<00:00,  7.88batches/s]
Training:   [Fold  2, Epoch  4, Loss: 0.000649, Acc: 0.8559, AUC: 0.8557]
Evaluation: [Fold  2, Epoch  4, Loss: 0.000615, Acc: 0.864

...

Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:09<00:00,  7.16batches/s]
Training:   [Fold  3, Epoch  1, Loss: 0.000732, Acc: 0.8412, AUC: 0.8007]
Evaluation: [Fold  3, Epoch  1, Loss: 0.000643, Acc: 0.8581, AUC: 0.8646]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:05<00:00,  7.64batches/s]
Training:   [Fold  3, Epoch  2, Loss: 0.000666, Acc: 0.8521, AUC: 0.8465]
Evaluation: [Fold  3, Epoch  2, Loss: 0.000627, Acc: 0.8618, AUC: 0.8702]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:06<00:00,  7.57batches/s]
Training:   [Fold  3, Epoch  3, Loss: 0.000656, Acc: 0.8544, AUC: 0.8523]
Evaluation: [Fold  3, Epoch  3, Loss: 0.000618, Acc: 0.8638, AUC: 0.8738]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:05<00:00,  7.59batches/s]
Training:   [Fold  3, Epoch  4, Loss: 0.000649, Acc: 0.8563, AUC: 0.8561]
Evaluation: [Fold  3, Epoch  4, Loss: 0.000613, Acc: 0.864

Using cache found in /home/ingo/.cache/torch/hub/facebookresearch_esm_main


Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:08<00:00,  7.32batches/s]
Training:   [Fold  4, Epoch  1, Loss: 0.000729, Acc: 0.8414, AUC: 0.8029]
Evaluation: [Fold  4, Epoch  1, Loss: 0.000638, Acc: 0.8579, AUC: 0.8647]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:08<00:00,  7.32batches/s]
Training:   [Fold  4, Epoch  2, Loss: 0.000667, Acc: 0.8520, AUC: 0.8457]
Evaluation: [Fold  4, Epoch  2, Loss: 0.000625, Acc: 0.8606, AUC: 0.8701]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:07<00:00,  7.42batches/s]
Training:   [Fold  4, Epoch  3, Loss: 0.000656, Acc: 0.8548, AUC: 0.8518]
Evaluation: [Fold  4, Epoch  3, Loss: 0.000616, Acc: 0.8625, AUC: 0.8736]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:06<00:00,  7.47batches/s]
Training:   [Fold  4, Epoch  4, Loss: 0.000650, Acc: 0.8565, AUC: 0.8553]
Evaluation: [Fold  4, Epoch  4, Loss: 0.000619, Acc: 0.862

Using cache found in /home/ingo/.cache/torch/hub/facebookresearch_esm_main


Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:11<00:00,  7.04batches/s]
Training:   [Fold  5, Epoch  1, Loss: 0.000729, Acc: 0.8417, AUC: 0.8029]
Evaluation: [Fold  5, Epoch  1, Loss: 0.000640, Acc: 0.8580, AUC: 0.8649]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:11<00:00,  6.96batches/s]
Training:   [Fold  5, Epoch  2, Loss: 0.000666, Acc: 0.8523, AUC: 0.8465]
Evaluation: [Fold  5, Epoch  2, Loss: 0.000628, Acc: 0.8601, AUC: 0.8707]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:11<00:00,  7.04batches/s]
Training:   [Fold  5, Epoch  3, Loss: 0.000654, Acc: 0.8549, AUC: 0.8527]
Evaluation: [Fold  5, Epoch  3, Loss: 0.000618, Acc: 0.8624, AUC: 0.8740]
Eval: 100%|███████████████████████████████████████████████████████| 500/500 [01:07<00:00,  7.37batches/s]
Training:   [Fold  5, Epoch  4, Loss: 0.000649, Acc: 0.8563, AUC: 0.8556]
Evaluation: [Fold  5, Epoch  4, Loss: 0.000613, Acc: 0.864

In [10]:
if 'model' in globals():
    del model
    print('deleted model')
if 'esm2' in globals():
    del esm2
    print('deleted esm2')

esm2, _ = torch.hub.load('facebookresearch/esm:main', 'esm2_t30_150M_UR50D')
model = ESM2BiLSTM(**params).to('cpu')
criterion = nn.BCEWithLogitsLoss()

# load best model, evaluate on test set
best_model = sorted(
    [f for f in os.listdir("../../params/c_term/ESM2BiLSTM/") if f.endswith(".pt")],
    reverse=True,
)[0]
print("Loaded model: ", best_model)
checkpoint = torch.load('../../params/c_term/ESM2BiLSTM/' + best_model, map_location='cpu')
model.load_state_dict(checkpoint)
model = model.to(device)
model.eval()

test_loss, test_acc, test_auc = process(model, test_loader, criterion)
print(
    f"Test Set Performance: Loss: {test_loss:.6f}, Acc: {test_acc:.4f}, AUC: {test_auc:.4f}"
)

print(
    f"Total model params: {total_model_params(model)}, trainable model params: {trainable_model_params(model)}"
)

deleted model
deleted esm2


Using cache found in /home/ingo/.cache/torch/hub/facebookresearch_esm_main


Loaded model:  auc0.8812_fold5_epoch10.pt
Eval: 100%|███████████████████████████████████████████████████████| 278/278 [00:36<00:00,  7.62batches/s]
Test Set Performance: Loss: 0.000597, Acc: 0.8687, AUC: 0.8834
Total model params: 152998267, trainable model params: 4858113
